In [1]:
#우선 파라미터 리스트를 불러오자

In [2]:
import pandas as pd
BestParam = pd.read_csv("BestParam.csv")

In [3]:
import rdkit
import numpy
import pandas
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
from rdkit.Chem import MACCSkeys

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from imblearn.combine import SMOTEENN 
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline 

In [4]:
selected_assay_list = ['NCCT_TPO_AUR_dn','NVS_ADME_hCYP1A2','NVS_ADME_hCYP2B6',
    'TOX21_AhR_LUC_Agonist', 'TOX21_Aromatase_Inhibition', 'TOX21_VDR_BLA_agonist_ratio', 'TOX21_VDR_BLA_antagonist_ratio', 'TOX21_AR_BLA_Agonist_ratio', 'TOX21_AR_BLA_Antagonist_ratio', 'TOX21_AR_LUC_MDAKB2_Agonist', 'TOX21_p53_BLA_p1_ratio', 'TOX21_p53_BLA_p2_ratio', 'TOX21_p53_BLA_p3_ratio', 'TOX21_p53_BLA_p4_ratio', 'TOX21_p53_BLA_p5_ratio', 'TOX21_HSE_BLA_agonist_ratio', 'TOX21_ELG1_LUC_Agonist', 'TOX21_ERa_BLA_Agonist_ratio', 'TOX21_ERa_BLA_Antagonist_ratio', 'TOX21_ESRE_BLA_ratio', 'TOX21_FXR_BLA_agonist_ratio', 'TOX21_FXR_BLA_antagonist_ratio', 'TOX21_GR_BLA_Agonist_ratio', 'TOX21_GR_BLA_Antagonist_ratio', 'TOX21_NFkB_BLA_agonist_ratio', 'TOX21_ARE_BLA_agonist_ratio', 'TOX21_TR_LUC_GH3_Agonist', 'TOX21_TR_LUC_GH3_Antagonist', 'TOX21_PPARg_BLA_Agonist_ratio', 'TOX21_PPARd_BLA_agonist_ratio', 'TOX21_PPARd_BLA_antagonist_ratio', 'TOX21_PPARg_BLA_antagonist_ratio']

In [5]:
selected_assay_list.remove('TOX21_NFkB_BLA_agonist_ratio') #active물질 개수가 3개뿐인 assay라 모델 

In [6]:
selected_assay_list = list(BestParam['Assays'])

In [7]:
selected_assay_list

['TOX21_VDR_BLA_agonist_ratio',
 'TOX21_VDR_BLA_antagonist_ratio',
 'TOX21_HSE_BLA_agonist_ratio',
 'TOX21_ESRE_BLA_ratio',
 'TOX21_FXR_BLA_agonist_ratio',
 'TOX21_FXR_BLA_antagonist_ratio',
 'TOX21_ARE_BLA_agonist_ratio',
 'TOX21_PPARd_BLA_agonist_ratio',
 'TOX21_PPARd_BLA_antagonist_ratio',
 'TOX21_PPARg_BLA_antagonist_ratio',
 'TOX21_ELG1_LUC_Agonist',
 'TOX21_AhR_LUC_Agonist',
 'TOX21_Aromatase_Inhibition',
 'TOX21_AR_BLA_Agonist_ratio',
 'TOX21_AR_BLA_Antagonist_ratio',
 'TOX21_AR_LUC_MDAKB2_Agonist',
 'TOX21_p53_BLA_p1_ratio',
 'TOX21_p53_BLA_p2_ratio',
 'TOX21_p53_BLA_p3_ratio',
 'TOX21_p53_BLA_p4_ratio',
 'TOX21_p53_BLA_p5_ratio',
 'TOX21_ERa_BLA_Agonist_ratio',
 'TOX21_ERa_BLA_Antagonist_ratio',
 'TOX21_GR_BLA_Agonist_ratio',
 'TOX21_GR_BLA_Antagonist_ratio',
 'TOX21_TR_LUC_GH3_Agonist',
 'TOX21_TR_LUC_GH3_Antagonist',
 'TOX21_PPARg_BLA_Agonist_ratio',
 'NVS_ADME_hCYP2B6',
 'NVS_ADME_hCYP1A2',
 'NCCT_TPO_AUR_dn']

In [8]:
assay_name = '335_all_assay'
df = pd.read_csv('./Data/'+assay_name+"_df.csv")

In [9]:
save_path = './31assays/'
!mkdir {save_path}
!mkdir {save_path+"dup_smiles/"}
!mkdir {save_path+"err_smiles/"}
!mkdir {save_path+"performance/"}
!mkdir {save_path+"params/"}

mkdir: cannot create directory ‘./31assays/’: File exists
mkdir: cannot create directory ‘./31assays/dup_smiles/’: File exists
mkdir: cannot create directory ‘./31assays/err_smiles/’: File exists
mkdir: cannot create directory ‘./31assays/performance/’: File exists
mkdir: cannot create directory ‘./31assays/params/’: File exists


In [10]:
def g_tf(assay_index) :
        ss = selected_assay_list[assay_index] # 그룹
        ffpp = "maccs"

        df_ss = df.loc[:,['smiles',ss]]
        df_ss = df_ss.dropna()
        assays_ss = df_ss.columns
        sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

            #Smiles --> MACCS Keys
        assay = ss
        df_train  = df_ss[['smiles',ss] ]

        smiles_list = []
        err_smiles = []# fingerfrint로 변환시 에러 smiles
        fps = []
        enc_y = []
        for index, row in df_train.iterrows():

          if row[-1] == 'N' : 
            target = 0
          else :
            target = 1

          try : 
            mol = Chem.MolFromSmiles(row['smiles'])

            if ffpp == 'maccs' :    
                fp = MACCSkeys.GenMACCSKeys(mol)
            elif ffpp == 'morgan' : 
                fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2)
            elif ffpp == 'rdkit' : 
                fp = Chem.RDKFingerprint(mol)
            elif ffpp == 'pattern' : 
                fp = Chem.rdmolops.PatternFingerprint(mol)
            elif ffpp == 'layerd' : 
                fp = Chem.rdmolops.LayeredFingerprint(mol)
            else : 
                print("fingerfrint selection error!")

            smiles_list.append(row['smiles'])   
            fps.append(fp)
            enc_y.append(target)
          except : 
            err_smiles.append(row['smiles'])
            pass

        np_fps = []
        for fp in fps:
          arr = numpy.zeros((1,))
          DataStructs.ConvertToNumpyArray(fp, arr)
          np_fps.append(arr)

        np_fps_array = numpy.array(np_fps)

        print("변환 에러난 smiles 개수 : ", len(err_smiles), err_smiles)

        x_train_res, y_train_res = np_fps_array, np.array(enc_y)
        train = pd.DataFrame(data=x_train_res)
        train['label'] = y_train_res

        d = pd.DataFrame(columns = ['smiles','Type'] )
        d['smiles'] = smiles_list
        d['Type'] = enc_y

        #모든 물질에 대한 fingerprint를 string형태로 바꾼다(비교하기위함).
        mychems = []
        for item in np_fps:
            #print(item)
            item = np.array(item,dtype=int) #왜 굳이 두번? int형으로 추가하려면 이렇게 하면 되는구나....
            s = item.tolist() #넘파이어레이를 리스트로 바꾼다. 
            t = [str(i) for i in s]
            mychems.append(''.join(t))

        i = 0
        counter = 0
        chemdict = {} #딗셔너리.
        listofdups = []
        for item in mychems:
            try:
                chemdict[str(item)].append(i)
                #print('dup found')
                for j in range(len(chemdict[str(item)])):
                    if (d['Type'][i] != d['Type'][chemdict[str(item)][j]]) :
                        listofdups.append(i)
                        counter+=1
            except: 
                #print('adding new')
                chemdict[str(item)] = []
                chemdict[str(item)].append(i)
            i+=1
        print(str(counter)+' '+str(len(list(set(listofdups)))))
        #smote적용하기 전에서 listofdups를 빼면 된다!

        dups_list = list(set(listofdups))
        pprint(dups_list) # 0인것만 지우는게 맞나?

        dups_list_0 = []
        for index in dups_list : 
          if train.iloc[index].label == 0 :
            dups_list_0.append(index)

        train = train.drop(dups_list_0)

        duplicated_smiles_0_df = pd.DataFrame()
        duplicated_smiles_0_df['deleted_duplicate_smiles_0'] = smiles_list
        duplicated_smiles_0_df = duplicated_smiles_0_df.iloc[dups_list_0,]
        duplicated_smiles_0_df

        save_path = './31assays/'
        dupdf = save_path+"dup_smiles/"+ss+"_dup_smiles_df.csv"
        duplicated_smiles_0_df.to_csv(dupdf)

        err_smiles_df = pd.DataFrame(columns = [ss], index = ['err_n','err_smiles'])
        err_smiles_df[ss] = [ len(err_smiles), err_smiles]

        ersdf = save_path+"err_smiles/"+ss+"_err_smiles_df.csv"
        err_smiles_df.to_csv(ersdf)


        skf = StratifiedKFold(n_splits=10)
        sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

        X = np.array( train.drop('label', axis =1) )
        y = np.array( train.label )
        #train test split
        X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y,
                                                            stratify=y, 
                                                            test_size=0.1,
                                                            random_state = 42)

        def evaluate(model, test_features, test_labels):
            predictions = model.predict(test_features)

            result = model.predict_proba(test_features) # 첫 클래스의 예측확률, 두번째 클래스의 예측 확률 어떤것이 0, 1이지?
            y_pred = model.predict(test_features)

            conf_mat = confusion_matrix(test_labels,y_pred,labels=[0,1])
            tn, fp, fn, tp = conf_mat.ravel()

            #auc_roc
            try:
                auc_roc =  roc_auc_score(test_labels,result[:,1] )
            except ValueError:
                auc_roc = 0

            #auc_pr
            try : 
                precision_l, recall_l, thresholds = precision_recall_curve(test_labels,result[:,1])
                auc_pr = auc(recall_l, precision_l)
                if np.isnan(auc_pr) : 
                    auc_pr = 0
            except : 
                auc_pr = 0


            #recall
            if (tp+fn) == 0 :
                recall = 0
            else : 
                recall = tp/(tp+fn)

            if np.isnan(recall) :
                recall = 0

            #precision
            if (tp+fp) == 0 :
                precision = 0
            else : 
                precision = tp/(tp+fp)

            if np.isnan(precision) :
                precision = 0

            #F1
            if recall == 0 or precision ==0  : 
                F1 = 0
            else : 
                F1 = 2* (precision * recall) / (precision + recall)

            if np.isnan(F1) : 
                F1 = 0 


            return auc_roc, auc_pr, recall, F1

        def Average(lst):
            return sum(lst) / len(lst)

        #dataframe to record performance
        result_df = pd.DataFrame(columns = ['AUC', 'AUC_PR', 'Recall','F1'])

        base_model = RandomForestClassifier(random_state = 42)

        # valid_base========================================================================================
        auc_list = []
        auc_pr_list = []
        recall_list = []
        F1_list = []

        for train_index, valid_index in skf.split(X_train_valid, y_train_valid): 

            X_train, X_valid = X[train_index], X[valid_index]
            y_train, y_valid = y[train_index], y[valid_index]

            if ss in ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn'] : 
                pass
            else : 
                X_train, y_train = sme.fit_sample(X_train, list(y_train) )

            clf = base_model
            clf.fit(X_train, y_train)
            auc_roc,auc_pr,recall,F1 = evaluate(clf, X_valid, y_valid)

            auc_list.append(auc_roc)
            auc_pr_list.append(auc_pr)
            recall_list.append(recall)
            F1_list.append(F1)

        result_df = result_df.append(pd.DataFrame([[Average(auc_list),Average(auc_pr_list),Average(recall_list),Average(F1_list)]], 
                                                  columns = ['AUC','AUC_PR','Recall','F1']) )


        #     #RF parameters

        #     n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
        #     max_features = ['auto', 'sqrt']
        #     max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
        #     max_depth.append(None)
        #     min_samples_split = [2, 5, 10]
        #     min_samples_leaf = [1, 2, 4]
        #     bootstrap = [True, False]
        #     random_grid = {'rf__n_estimators': n_estimators,
        #                    'rf__max_features': max_features,
        #                    'rf__max_depth': max_depth,
        #                    'rf__min_samples_split': min_samples_split,
        #                    'rf__min_samples_leaf': min_samples_leaf,
        #                    'rf__bootstrap': bootstrap}

        #     pipeline = Pipeline([("smoteenn", sme), ("rf", base_model)])
        #     rf_random = RandomizedSearchCV(estimator = pipeline, param_distributions = random_grid, n_iter = 100, cv = 10, 
        #                                scoring='f1',verbose=2, random_state=42, n_jobs = -1)

        #     # Fit the random search model
        #     rf_random.fit(X_train_valid, y_train_valid)

        #     df_params = pd.DataFrame([rf_random.best_params_])
        #     df_params.index = [ss]
        #     df_params.to_csv(save_path+'params/'+ss+".csv")



        params_rf = dict(BestParam.loc[assay_index][1:])
        if params_rf['max_depth'] != 'None' : 
            params_rf['max_depth'] = int(params_rf['max_depth'])
        else : 
            params_rf['max_depth'] = None
        rf_random = RandomForestClassifier(random_state = 42).set_params(**params_rf)


        #valid_tuned========================================================================================
        auc_list = []
        auc_pr_list = []
        recall_list = []
        F1_list = []

        def Average(lst):
            return sum(lst) / len(lst)

        for train_index, valid_index in skf.split(X_train_valid, y_train_valid): 

            X_train, X_valid = X[train_index], X[valid_index]
            y_train, y_valid = y[train_index], y[valid_index]

            if ss in ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn'] : 
                pass
            else : 
                X_train, y_train = sme.fit_sample(X_train, list(y_train) )

            clf = RandomForestClassifier(random_state = 42)
            clf.set_params(**params_rf)
            clf.fit(X_train, y_train)
            auc_roc,auc_pr,recall,F1 = evaluate(clf, X_valid, y_valid)

            auc_list.append(auc_roc)
            auc_pr_list.append(auc_pr)
            recall_list.append(recall)
            F1_list.append(F1)

        result_df = result_df.append(pd.DataFrame([[Average(auc_list),Average(auc_pr_list),Average(recall_list),Average(F1_list)]], columns = ['AUC','AUC_PR','Recall','F1']) )

        X_train_valid, y_train_valid = sme.fit_sample(X_train_valid, y_train_valid )

        #test_base ========================================================================================
        base_model = RandomForestClassifier(random_state = 42)
        base_model.fit(X_train_valid, y_train_valid)
        auc_roc,auc_pr, recall, F1 = evaluate(base_model, X_test, y_test)
        result_df = result_df.append(pd.DataFrame([[auc_roc,auc_pr,recall,F1]], columns = ['AUC','AUC_PR','Recall','F1']) )

        #test_tuned
        best_random = RandomForestClassifier(random_state = 42).set_params(**params_rf)
        best_random.fit(X_train_valid, y_train_valid)
        auc_t, auc_pr_t, recall_t, F1_t = evaluate(best_random, X_test, y_test)
        result_df = result_df.append(pd.DataFrame([[auc_t,auc_pr_t,recall_t,F1_t]], columns = ['AUC','AUC_PR','Recall','F1']) )

        # error.. ZeroDivisionError: division by zero
        #     print('Improvement of {:0.2f}%.'.format( 100 * (auc_t - auc) / auc))
        #     print('Improvement of {:0.2f}%.'.format( 100 * (recall_t - recall) / recall))
        #     print('Improvement of {:0.2f}%.'.format( 100 * (F1_t - F1) / F1))

        result_df.index = ['valid_base', 'valid_tuned', 'test_base', 'test_tuned']
        result_df.index.name = ss
        result_df.to_csv(save_path+'performance/performance'+ss+".csv")

In [12]:
ss

'TOX21_ESRE_BLA_ratio'

In [13]:
best_random

RandomForestClassifier(max_features='sqrt', random_state=42)

In [14]:
result_df

,AUC,AUC_PR,Recall,F1
TOX21_ESRE_BLA_ratio,,,,
valid_base,0.733948,0.299513,0.225000,0.294545
valid_tuned,0.733948,0.299513,0.225000,0.294545
test_base,0.926357,0.671942,0.666667,0.800000
test_tuned,0.926357,0.671942,0.666667,0.800000


In [ ]:
assay_index = 3
ss = selected_assay_list[assay_index] # 그룹
ffpp = "maccs"

df_ss = df.loc[:,['smiles',ss]]
df_ss = df_ss.dropna()
assays_ss = df_ss.columns
sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

    #Smiles --> MACCS Keys
assay = ss
df_train  = df_ss[['smiles',ss] ]

smiles_list = []
err_smiles = []# fingerfrint로 변환시 에러 smiles
fps = []
enc_y = []
for index, row in df_train.iterrows():

  if row[-1] == 'N' : 
    target = 0
  else :
    target = 1

  try : 
    mol = Chem.MolFromSmiles(row['smiles'])

    if ffpp == 'maccs' :    
        fp = MACCSkeys.GenMACCSKeys(mol)
    elif ffpp == 'morgan' : 
        fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    elif ffpp == 'rdkit' : 
        fp = Chem.RDKFingerprint(mol)
    elif ffpp == 'pattern' : 
        fp = Chem.rdmolops.PatternFingerprint(mol)
    elif ffpp == 'layerd' : 
        fp = Chem.rdmolops.LayeredFingerprint(mol)
    else : 
        print("fingerfrint selection error!")

    smiles_list.append(row['smiles'])   
    fps.append(fp)
    enc_y.append(target)
  except : 
    err_smiles.append(row['smiles'])
    pass

np_fps = []
for fp in fps:
  arr = numpy.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  np_fps.append(arr)

np_fps_array = numpy.array(np_fps)

print("변환 에러난 smiles 개수 : ", len(err_smiles), err_smiles)

x_train_res, y_train_res = np_fps_array, np.array(enc_y)
train = pd.DataFrame(data=x_train_res)
train['label'] = y_train_res

d = pd.DataFrame(columns = ['smiles','Type'] )
d['smiles'] = smiles_list
d['Type'] = enc_y

#모든 물질에 대한 fingerprint를 string형태로 바꾼다(비교하기위함).
mychems = []
for item in np_fps:
    #print(item)
    item = np.array(item,dtype=int) #왜 굳이 두번? int형으로 추가하려면 이렇게 하면 되는구나....
    s = item.tolist() #넘파이어레이를 리스트로 바꾼다. 
    t = [str(i) for i in s]
    mychems.append(''.join(t))

i = 0
counter = 0
chemdict = {} #딗셔너리.
listofdups = []
for item in mychems:
    try:
        chemdict[str(item)].append(i)
        #print('dup found')
        for j in range(len(chemdict[str(item)])):
            if (d['Type'][i] != d['Type'][chemdict[str(item)][j]]) :
                listofdups.append(i)
                counter+=1
    except: 
        #print('adding new')
        chemdict[str(item)] = []
        chemdict[str(item)].append(i)
    i+=1
print(str(counter)+' '+str(len(list(set(listofdups)))))
#smote적용하기 전에서 listofdups를 빼면 된다!

dups_list = list(set(listofdups))
pprint(dups_list) # 0인것만 지우는게 맞나?

dups_list_0 = []
for index in dups_list : 
  if train.iloc[index].label == 0 :
    dups_list_0.append(index)

train = train.drop(dups_list_0)

duplicated_smiles_0_df = pd.DataFrame()
duplicated_smiles_0_df['deleted_duplicate_smiles_0'] = smiles_list
duplicated_smiles_0_df = duplicated_smiles_0_df.iloc[dups_list_0,]
duplicated_smiles_0_df

save_path = './31assays/'
dupdf = save_path+"dup_smiles/"+ss+"_dup_smiles_df.csv"
duplicated_smiles_0_df.to_csv(dupdf)

err_smiles_df = pd.DataFrame(columns = [ss], index = ['err_n','err_smiles'])
err_smiles_df[ss] = [ len(err_smiles), err_smiles]

ersdf = save_path+"err_smiles/"+ss+"_err_smiles_df.csv"
err_smiles_df.to_csv(ersdf)


skf = StratifiedKFold(n_splits=10)
sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

X = np.array( train.drop('label', axis =1) )
y = np.array( train.label )
#train test split
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.1,
                                                    random_state = 42)

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)

    result = model.predict_proba(test_features) # 첫 클래스의 예측확률, 두번째 클래스의 예측 확률 어떤것이 0, 1이지?
    y_pred = model.predict(test_features)

    conf_mat = confusion_matrix(test_labels,y_pred,labels=[0,1])
    tn, fp, fn, tp = conf_mat.ravel()

    #auc_roc
    try:
        auc_roc =  roc_auc_score(test_labels,result[:,1] )
    except ValueError:
        auc_roc = 0

    #auc_pr
    try : 
        precision_l, recall_l, thresholds = precision_recall_curve(test_labels,result[:,1])
        auc_pr = auc(recall_l, precision_l)
        if np.isnan(auc_pr) : 
            auc_pr = 0
    except : 
        auc_pr = 0


    #recall
    if (tp+fn) == 0 :
        recall = 0
    else : 
        recall = tp/(tp+fn)

    if np.isnan(recall) :
        recall = 0

    #precision
    if (tp+fp) == 0 :
        precision = 0
    else : 
        precision = tp/(tp+fp)

    if np.isnan(precision) :
        precision = 0

    #F1
    if recall == 0 or precision ==0  : 
        F1 = 0
    else : 
        F1 = 2* (precision * recall) / (precision + recall)

    if np.isnan(F1) : 
        F1 = 0 


    return auc_roc, auc_pr, recall, F1

def Average(lst):
    return sum(lst) / len(lst)

#dataframe to record performance
result_df = pd.DataFrame(columns = ['AUC', 'AUC_PR', 'Recall','F1'])

base_model = RandomForestClassifier(random_state = 42)

# valid_base========================================================================================
auc_list = []
auc_pr_list = []
recall_list = []
F1_list = []

for train_index, valid_index in skf.split(X_train_valid, y_train_valid): 

    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    if ss in ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn'] : 
        pass
    else : 
        X_train, y_train = sme.fit_sample(X_train, list(y_train) )

    clf = base_model
    clf.fit(X_train, y_train)
    auc_roc,auc_pr,recall,F1 = evaluate(clf, X_valid, y_valid)

    auc_list.append(auc_roc)
    auc_pr_list.append(auc_pr)
    recall_list.append(recall)
    F1_list.append(F1)

result_df = result_df.append(pd.DataFrame([[Average(auc_list),Average(auc_pr_list),Average(recall_list),Average(F1_list)]], 
                                          columns = ['AUC','AUC_PR','Recall','F1']) )


#     #RF parameters

#     n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#     max_features = ['auto', 'sqrt']
#     max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#     max_depth.append(None)
#     min_samples_split = [2, 5, 10]
#     min_samples_leaf = [1, 2, 4]
#     bootstrap = [True, False]
#     random_grid = {'rf__n_estimators': n_estimators,
#                    'rf__max_features': max_features,
#                    'rf__max_depth': max_depth,
#                    'rf__min_samples_split': min_samples_split,
#                    'rf__min_samples_leaf': min_samples_leaf,
#                    'rf__bootstrap': bootstrap}

#     pipeline = Pipeline([("smoteenn", sme), ("rf", base_model)])
#     rf_random = RandomizedSearchCV(estimator = pipeline, param_distributions = random_grid, n_iter = 100, cv = 10, 
#                                scoring='f1',verbose=2, random_state=42, n_jobs = -1)

#     # Fit the random search model
#     rf_random.fit(X_train_valid, y_train_valid)

#     df_params = pd.DataFrame([rf_random.best_params_])
#     df_params.index = [ss]
#     df_params.to_csv(save_path+'params/'+ss+".csv")



params_rf = dict(BestParam.loc[assay_index][1:])
if params_rf['max_depth'] != 'None' : 
    params_rf['max_depth'] = int(params_rf['max_depth'])
else : 
    params_rf['max_depth'] = None
rf_random = RandomForestClassifier(random_state = 42).set_params(**params_rf)


#valid_tuned========================================================================================
auc_list = []
auc_pr_list = []
recall_list = []
F1_list = []

def Average(lst):
    return sum(lst) / len(lst)

for train_index, valid_index in skf.split(X_train_valid, y_train_valid): 

    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    if ss in ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn'] : 
        pass
    else : 
        X_train, y_train = sme.fit_sample(X_train, list(y_train) )

    clf = RandomForestClassifier(random_state = 42)
    clf.set_params(**params_rf)
    clf.fit(X_train, y_train)
    auc_roc,auc_pr,recall,F1 = evaluate(clf, X_valid, y_valid)

    auc_list.append(auc_roc)
    auc_pr_list.append(auc_pr)
    recall_list.append(recall)
    F1_list.append(F1)

result_df = result_df.append(pd.DataFrame([[Average(auc_list),Average(auc_pr_list),Average(recall_list),Average(F1_list)]], columns = ['AUC','AUC_PR','Recall','F1']) )

X_train_valid, y_train_valid = sme.fit_sample(X_train_valid, y_train_valid )

#test_base ========================================================================================
base_model = RandomForestClassifier(random_state = 42)
base_model.fit(X_train_valid, y_train_valid)
auc_roc,auc_pr, recall, F1 = evaluate(base_model, X_test, y_test)
result_df = result_df.append(pd.DataFrame([[auc_roc,auc_pr,recall,F1]], columns = ['AUC','AUC_PR','Recall','F1']) )

#test_tuned
best_random = RandomForestClassifier(random_state = 42).set_params(**params_rf)
best_random.fit(X_train_valid, y_train_valid)
auc_t, auc_pr_t, recall_t, F1_t = evaluate(best_random, X_test, y_test)
result_df = result_df.append(pd.DataFrame([[auc_t,auc_pr_t,recall_t,F1_t]], columns = ['AUC','AUC_PR','Recall','F1']) )

# error.. ZeroDivisionError: division by zero
#     print('Improvement of {:0.2f}%.'.format( 100 * (auc_t - auc) / auc))
#     print('Improvement of {:0.2f}%.'.format( 100 * (recall_t - recall) / recall))
#     print('Improvement of {:0.2f}%.'.format( 100 * (F1_t - F1) / F1))

result_df.index = ['valid_base', 'valid_tuned', 'test_base', 'test_tuned']
result_df.index.name = ss
result_df.to_csv(save_path+'performance/performance'+ss+".csv")

In [15]:
result_df = pd.DataFrame(columns = ['AUC''AUC_PR','Recall','F1'])

In [16]:
assays_index_list = [i for i in range(31)]

In [17]:
from rdkit import RDLogger                                                                                                                                                               
RDLogger.DisableLog('rdApp.*') 

In [18]:
import math 
import time 
import datetime 
start = time.time() 

In [19]:
import multiprocessing
CPU_CORE = multiprocessing.cpu_count() # 멀티프로세싱 CPU 사용 수
if __name__=='__main__':
    pool = multiprocessing.Pool(processes=CPU_CORE)
    pool.map(g_tf, assays_index_list)
    pool.close()
    pool.join()  

변환 에러난 smiles 개수 :  0 []
변환 에러난 smiles 개수 :  0 00
[]
 []
1 1
[65]
변환 에러난 smiles 개수 :  2 ['CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1']
27 18
[193,
 354,
 545,
 196,
 388,
 358,
 383,
 516,
 546,
 508,
 464,
 189,
 505,
 378,
 347,
 316,
 221,
 351]
변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
28 26
[6795,
 3476,
 6814,
 6435,
 5028변환 에러난 smiles 개수 : ,
  401610,
  ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O

/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
Process ForkPoolWorker-5:
Process ForkPoolWorker-31:
Process ForkPoolWorker-35:
Process ForkPoolWorker-33:
Process ForkPoolWorker-38:
Process ForkPoolWorker-47:
  File "/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
Process ForkPoolWorker-40:
Process ForkPoolWorker-36:
Process ForkPoolWorker-43:
Process ForkPoolWorker-44:
Process ForkPoolWorker-42:
Process ForkPoolWorker-34:
Process ForkPoolWorker-41:
Process ForkPoolWorker-39:
Process ForkPoolWorker-37:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (mos

KeyboardInterrupt: 

Process ForkPoolWorker-28:
Traceback (most recent call last):
Process ForkPoolWorker-25:
  File "/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/tmp/ipykernel_20145/2058322927.py", line 209, in g_tf
    X_train, y_train = sme.fit_sample(X_train, list(y_train) )
  File "/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/imblearn/base.py", line 83, in fit_resample
    output = self._fit_resample(X, y)
  File "/home2/prof/chemdsrc

In [ ]:
end = time.time() 
sec = (end - start) 
result = datetime.timedelta(seconds=sec) 
print(result) 
result_list = str(datetime.timedelta(seconds=sec)).split(".") 
print(result_list[0])

#우선.. 8시간 17분 소요

In [ ]:
result_df

In [ ]:
import os
finished_list = os.listdir("./31assays/performance")

In [ ]:
finished_assays=[]
for i in finished_list : 
    i2 = i.replace("performance","").replace(".csv","")
    finished_assays.append(i2)

In [ ]:
not_finished_assays = list(set(selected_assay_list).difference(set(finished_assays)))

In [ ]:
not_finished_assays

In [ ]:
not_finished_index =[]
for i in not_finished_assays : 
    not_finished_index.append( selected_assay_list.index(i) )

In [ ]:
not_finished_index

In [ ]:
import multiprocessing
CPU_CORE = multiprocessing.cpu_count() # 멀티프로세싱 CPU 사용 수
if __name__=='__main__':
    pool = multiprocessing.Pool(processes=CPU_CORE)
    pool.map(g_tf, not_finished_index)
    pool.close()
    pool.join()  

In [ ]:
!pwd

In [ ]:
selected_assay_list.index('TOX21_VDR_BLA_agonist_ratio')

In [ ]:
for i in not_finished_assays : 
    g_tf(selected_assay_list.index(i))

In [ ]:
g_tf(selected_assay_list.index('NVS_ADME_hCYP1A2'))

In [ ]:
g_tf(selected_assay_list.index('TOX21_PPARd_BLA_antagonist_ratio'))

In [ ]:
g_tf(selected_assay_list.index('TOX21_VDR_BLA_agonist_ratio'))

In [ ]:
g_tf(selected_assay_list.index('TOX21_ELG1_LUC_Agonist'))

In [ ]:
g_tf(selected_assay_list.index('TOX21_TR_LUC_GH3_Agonist'))

In [ ]:
g_tf(selected_assay_list.index('TOX21_ESRE_BLA_ratio'))

In [ ]:
g_tf(selected_assay_list.index('TOX21_FXR_BLA_agonist_ratio'))

In [ ]:
end = time.time() 
sec = (end - start) 
result = datetime.timedelta(seconds=sec) 
print(result) 
result_list = str(datetime.timedelta(seconds=sec)).split(".") 
print(result_list[0])

#우선.. 8시간 17분 소요

In [ ]:
os.listdir('')

In [ ]:
# for i in assays_index_list :
#     g_tf(i)